In [2]:
import sys
sys.path
sys.path.append('/home/helge/code/back_of_the_envelope')

import pandas as pd
import random
import json

from forecasters import get_forecaster, forecast_election

forecast = {}

for region in ["Hessen", "Bayern", "Bremen"]:
    fc = get_forecaster(region)
    forecast[region] = forecast_election(fc)
    print(region)
    print(forecast[region])
    print()

with open("election_data_20221026.json", "w") as f:
    json.dump(forecast, f)


Hessen
{'Chance to be in parliament': [{'name': 'CDU', 'percentage': 100.0}, {'name': 'SPD', 'percentage': 100.0}, {'name': 'GRÜNE', 'percentage': 100.0}, {'name': 'AfD', 'percentage': 100.0}, {'name': 'FDP', 'percentage': 83.5}], 'Probability of Government': [{'name': 'CDU GRÜNE', 'percentage': 80.5}, {'name': 'SPD GRÜNE', 'percentage': 11.8}, {'name': 'GRÜNE CDU', 'percentage': 5.7}]}

Bayern
{'Chance to be in parliament': [{'name': 'CSU', 'percentage': 100.0}, {'name': 'SPD', 'percentage': 100.0}, {'name': 'GRÜNE', 'percentage': 100.0}, {'name': 'FW', 'percentage': 100.0}, {'name': 'AfD', 'percentage': 100.0}, {'name': 'FDP', 'percentage': 87.4}], 'Probability of Government': [{'name': 'CSU FW', 'percentage': 51.0}, {'name': 'CSU GRÜNE', 'percentage': 37.6}, {'name': 'CSU FDP', 'percentage': 8.7}]}

Bremen
{'Chance to be in parliament': [{'name': 'CDU', 'percentage': 100.0}, {'name': 'SPD', 'percentage': 100.0}, {'name': 'GRÜNE', 'percentage': 100.0}, {'name': 'LINKE', 'percentage':

In [2]:
from forecasters.election import ElectionForecaster
from forecasters import forecast_election

bremen_forecaster = ElectionForecaster(polling_data)
bremen_forecaster.sample()

forecast_election(bremen_forecaster)

{'in_parliament': [{'name': 'CDU', 'percentage': 100.0},
  {'name': 'SPD', 'percentage': 100.0},
  {'name': 'GRÜNE', 'percentage': 100.0},
  {'name': 'LINKE', 'percentage': 99.9},
  {'name': 'AfD', 'percentage': 85.5},
  {'name': 'FDP', 'percentage': 83.8}],
 'government': [{'name': 'SPD GRÜNE', 'percentage': 82.0},
  {'name': 'SPD CDU', 'percentage': 10.2},
  {'name': 'CDU GRÜNE', 'percentage': 4.3},
  {'name': 'GRÜNE CDU', 'percentage': 2.3}]}

In [4]:
df = pd.read_html('https://www.wahlrecht.de/umfragen/landtage/bayern.htm')[1]
row = df.iloc[0]
parties = ['CSU',
       'SPD', 'GRÜNE', 'FDP', 'LINKE', 'FW', 'AfD', 'Sonstige']
polling_data = {
    p: int(row[p].split(" ")[0])
       for p in parties
}
bayern_forecaster = ElectionForecaster(polling_data)

in_parliament = {p: 0 for p in parties}

number_red_green = 0

for x in bayern_forecaster.predict(1000):
    for p in x.keys():
        in_parliament[p] += 1
    for p, v in x.items():
        if v > 50:
            print(f"{p} has majority")

    if x["SPD"] + x["GRÜNE"] > 50:
        number_red_green += 1

print(in_parliament)
print(number_red_green)

CSU has majority
CSU has majority
CSU has majority
{'CSU': 1000, 'SPD': 1000, 'GRÜNE': 1000, 'FDP': 853, 'LINKE': 0, 'FW': 1000, 'AfD': 1000, 'Sonstige': 0}
0


In [5]:
from itertools import combinations

in_parliament = {p: 0 for p in parties}
majorities = {}

number_red_green = 0

for x in bayern_forecaster.predict(1000):
    for p in x.keys():
        in_parliament[p] += 1
    for p, v in x.items():
        if v > 50:
            print(f"{p} has majority")

    pip = list(x.keys())

    for a, b in combinations(pip, 2):
        if x[a] + x[b] > 50:
            if f"{a}_{b}" in majorities:
                majorities[f"{a}_{b}"] += 1
            else:
                majorities[f"{a}_{b}"] = 1

print(in_parliament)
print(majorities)

CSU has majority
CSU has majority
CSU has majority
{'CSU': 1000, 'SPD': 1000, 'GRÜNE': 1000, 'FDP': 863, 'LINKE': 0, 'FW': 1000, 'AfD': 1000, 'Sonstige': 0}
{'CSU_SPD': 845, 'CSU_GRÜNE': 1000, 'CSU_FW': 708, 'CSU_AfD': 847, 'CSU_FDP': 199}
